In [ ]:
#required packages

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
import PIL
import cv2
import shutil
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping

In [ ]:
#importing the image dataset

os.mkdir('./images/')
alpha = 'a'
for i in range(0, 26): 
    os.mkdir('./images/' + alpha)
    alpha = chr(ord(alpha) + 1)

for file in os.listdir(rootdir):
    letter = file[0]
    try:
        shutil.copy(rootdir+file, './images/' + letter + '/' + file)
    except :
        print(file)
        
datagen = ImageDataGenerator(rotation_range=20,
                             shear_range=10,
                             validation_split=0.2)

train_generator = datagen.flow_from_directory('./images/',
                                              target_size=(28,28),
                                              subset='training')

val_generator = datagen.flow_from_directory('./images/',
                                            target_size=(28,28),
                                            subset='validation')

In [ ]:
#model training

model_ckpt = ModelCheckpoint('BrailleNet.h5',save_best_only=True)
reduce_lr = ReduceLROnPlateau(patience=8,verbose=0)
early_stop = EarlyStopping(patience=15,verbose=1)

entry = layers.Input(shape=(28,28,3))
x = layers.SeparableConv2D(64,(3,3),activation='relu')(entry)
x = layers.MaxPooling2D((2,2))(x)
x = layers.SeparableConv2D(128,(3,3),activation='relu')(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.SeparableConv2D(256,(2,2),activation='relu')(x)
x = layers.GlobalMaxPooling2D()(x)
x = layers.Dense(256)(x)
x = layers.LeakyReLU()(x)
x = layers.Dense(64,kernel_regularizer=l2(2e-4))(x)
x = layers.LeakyReLU()(x)
x = layers.Dense(26,activation='softmax')(x)

model = Model(entry,x)

In [ ]:
#optimiser 

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#testing and predicting the model

history = model.fit_generator(train_generator,
                              validation_data=val_generator,
                              epochs=50,
                              callbacks=[model_ckpt,reduce_lr,early_stop],
                              verbose=1)